# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from datetime import datetime

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, pipeline
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import AutoModelForCausalLM, AutoModel
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.llms import OpenAI

# Getting from csv and splitting

In [3]:
csv_file_path = 'clean_example_data.csv'

loader = CSVLoader(file_path=csv_file_path)

data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

docs = text_splitter.split_documents(data)

In [5]:
docs[0]

Document(metadata={'source': 'clean_example_data.csv', 'row': 0}, page_content='content: ODTÜ 2022-2023 yılı sonbahar dönemi için etkileşimli kayıt sistemi 28 Eylül 2022 tarihinde, saat 09.00’da öğrencilere açılacak. Birer saat arayla, başta 4. sınıflar olmak üzere tüm sınıflara kademeli olarak açılmaya devam edecek. Etkileşimli kayıtlar 30 Eylül, saat 17.00’de sona erecektir. Danışman onayları ise 28 Eylül’de başlayıp 30 Eylül saat 23.59’da sona erecektir. Not: Kayıt programına kampüs dışından erişimlerde VPN bağlantısı şartı bulunmamaktadır.')

# Getting an pre-trained embedding model

In [6]:
model_path = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name = model_path,      
    model_kwargs = model_kwargs,  
    encode_kwargs = encode_kwargs 
)

In [7]:
text = 'Üreten ekibe hoş geldin'
query_result = embeddings.embed_query(text)
len(query_result)

384

# Creating a vector database and a retriever

In [8]:
vector_db = FAISS.from_documents(docs, embeddings)

In [9]:
retriever = vector_db.as_retriever()

docs = retriever.invoke("ders nasıl seçilir")
print(docs)

[Document(metadata={'source': 'clean_example_data.csv', 'row': 3}, page_content='5. Bu kısım dersleri eklediğimiz ana alan diyebiliriz. Ders seçimi hikayemizin kaderini belirleyen alan da tam olarak burası oluyor.\n“Course Code” kısmına almak istediğimiz dersin kodunu, “Course Section” kısmına ise dersin section’ını yazacağız. Altından ise dersin türünü seçmeniz gerekiyor. Alacağınız ders must ise must seçeneğini değil ise free seçeneğini işaretleyebilirsiniz. Geriye sadece robot olmadığınızı ispatlamak ve dersi eklemek kalıyor. Eğer şanslı bir ODTÜ öğrencisiyseniz tek seferde robot olmadığınızı ispatlayabilirsiniz. 6. Bu alan pandemi ile birlikte hayatımıza giren bir kısım. Yüz yüze eğitime geçiyor olsak da danışman onayımızı buradaki butona basarak online olarak yapabiliyoruz. Bu alana tıkladıktan sonra değişiklik yapamamaktayız bu yüzden aldığınız derslerden emin olmadan bu butona tıklamamalısınız.'), Document(metadata={'source': 'clean_example_data.csv', 'row': 1}, page_content='çı

# Llama 3.2 3b

In [10]:
llama_llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.1,
)

# Opus en-tr model

In [11]:
en_tr_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-tr", device="cuda")

In [12]:
docs = vector_db.similarity_search(query=question,k=3)

NameError: name 'question' is not defined

In [ ]:
pre_prompt = "Answer the following question in English using the information above. Do not add any unnecessary information and act as a personal assistant"

In [ ]:
prompt = f"{docs[0].page_content}\n{docs[1].page_content}\n{docs[2].page_content}\n{pre_prompt}\n{question}"

# Creating and calculating the speed of the chain

In [15]:
question = input("Ask a question (in Turkish):  ")

Ask a question (in Turkish):   Yaz okulu nedir


In [16]:
pre_prompt = "Answer the following question in English using the information above. Do not add any unnecessary information and act as a personal assistant"

In [17]:
time_0 = datetime.now()

docs = vector_db.similarity_search(query=question,k=3)
prompt = f"{docs[0].page_content}\n{docs[1].page_content}\n{docs[2].page_content}\n{pre_prompt}\n{question}"

time_1 = datetime.now()

en_answer = llama_llm.invoke(prompt).content

time_2 = datetime.now()

tr_answer = en_tr_pipe(en_answer)[0]["translation_text"]

time_3 = datetime.now()

print(tr_answer)

Yaz Okulu, ayrı bir dönem yerine bir önceki bahar döneminin uzantısı olarak kabul edilen ODTÜ'nün yaz dönemidir. Genel not ortalamanızı veya tamamladığınız dönem sayısını etkilemez. Yaz Okulu'da alabileceğiniz ders sayısı dönem başına 2 ile sınırlıdır.


In [18]:
prompt_creating_time = time_1 - time_0
llama_gen_time = time_2 - time_1
opus_translate_time = time_3 - time_2
overall_time = time_3 - time_0

print(f"Similarity search + prompt creating time: {prompt_creating_time}\nLlama answer generation time:             {llama_gen_time}\nOpus en-tr translation time:              {opus_translate_time}\nOverall execution time:                   {overall_time}")

Similarity search + prompt creating time: 0:00:00.017879
Llama answer generation time:             0:00:01.418441
Opus en-tr translation time:              0:00:00.609412
Overall execution time:                   0:00:02.045732
